In [1]:
using Revise
using Catan
using CatanLearning
using Random
using CSV
using MLJ
using DataFrames
import DataFramesMeta as DFM
using DelimitedFiles
import MLJModelInterface
const MMI = MLJModelInterface

[ Info: Registering DefaultRobotPlayer
[ Info: Registering HumanPlayer
┌ Info: Registering EmpathRobotPlayer
└ @ Catan /home/bryan/Projects/Catan/CatanEngine.jl/src/players/structs.jl:120
┌ Info: Registering MutatedEmpathRobotPlayer
└ @ Catan /home/bryan/Projects/Catan/CatanEngine.jl/src/players/structs.jl:120
┌ Info: Registering TemporalDifferencePlayer
└ @ Catan /home/bryan/Projects/Catan/CatanEngine.jl/src/players/structs.jl:120


MLJModelInterface

In [2]:
function naive_model_proba(X_in)
    X = X_in[1]
    #println("analyzing $X")
    unscaled_feature = X[!,:CountSettlement] + X[!,:CountCity] + 2*(X[!,:HasLargestArmy] + X[!,:HasLongestRoad])
    return min.(unscaled_feature ./ 10, 1.0)
end

MMI.@mlj_model mutable struct NaivePublicVictoryPoints <: MLJModelInterface.Deterministic
end

MMI.fit(::NaivePublicVictoryPoints, verbosity, data...) = (nothing, nothing, nothing)
MMI.predict(model::NaivePublicVictoryPoints, fitresult, new_data...) = naive_model_proba(new_data)

model = NaivePublicVictoryPoints()
naive_machine = machine(model, nothing, nothing) |> MLJ.fit!

┌ Info: Training machine(NaivePublicVictoryPoints(), …).
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


trained Machine; caches model-specific representations of data
  model: NaivePublicVictoryPoints()
  args: 
    1:	Source @066 ⏎ Nothing
    2:	Source @759 ⏎ Nothing


In [27]:
config_file = "configs/feature_generation.toml"
(configs, player_configs, logger) = Catan.parse_configs(config_file)
rf_machine = CatanLearning.try_load_public_model_from_csv(:any, configs)
#rf_machine = CatanLearning.train_model_from_csv(CatanLearning.load_tree_model(), player_configs["PUBLIC_FEATURES"], num_tuning_iterations = 1)

Savefile set to savefile.txt
Configs loaded from configs/feature_generation.toml


trained Machine; caches model-specific representations of data
  model: RandomForestClassifier(max_depth = -1, …)
  args: 


In [35]:
test_data_csv = "./data/public_features.csv"
df = CatanLearning.load_typed_features_from_csv(test_data_csv)
y, X = unpack(df, ==(:WonGame));
typeof(X)
for col in names(X)
    println(col)
end

CountSettlement
CountTotalSettlement
CountCity
CountRoad
SumWoodDiceWeight
SumBrickDiceWeight
SumPastureDiceWeight
SumStoneDiceWeight
SumGrainDiceWeight
CountPortWood
CountPortBrick
CountPortPasture
CountPortStone
CountPortGrain
CountTotalWood
CountTotalBrick
CountTotalPasture
CountTotalStone
CountTotalGrain
CountDevCardsKnight
CountDevCardsMonopoly
CountDevCardsYearOfPlenty
CountDevCardsRoadBuilding
CountDevCardsVictoryPoint
HasLargestArmy
HasLongestRoad


In [36]:
test_data_csv = "./data/public_features.csv"
df = CatanLearning.load_typed_features_from_csv(test_data_csv)

Row,CountSettlement,CountTotalSettlement,CountCity,CountRoad,SumWoodDiceWeight,SumBrickDiceWeight,SumPastureDiceWeight,SumStoneDiceWeight,SumGrainDiceWeight,CountPortWood,CountPortBrick,CountPortPasture,CountPortStone,CountPortGrain,CountTotalWood,CountTotalBrick,CountTotalPasture,CountTotalStone,CountTotalGrain,CountDevCardsKnight,CountDevCardsMonopoly,CountDevCardsYearOfPlenty,CountDevCardsRoadBuilding,CountDevCardsVictoryPoint,HasLargestArmy,HasLongestRoad,WonGame
,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Bool,Bool,Cat…
1,2,2,0,3,0,7,0,3,0,0,0,0,0,0,8,8,2,4,2,0,0,0,0,0,false,false,false
2,2,2,0,5,0,0,5,3,0,0,0,0,0,1,7,7,13,5,6,1,0,0,1,0,false,false,false
3,2,5,3,14,8,20,2,20,8,0,0,0,0,0,19,19,4,21,10,0,0,1,0,0,false,true,true
4,2,2,0,4,6,0,0,3,0,0,0,0,0,0,6,6,4,4,4,1,0,0,0,0,false,false,false
5,4,4,0,14,8,7,0,9,1,0,0,0,0,0,25,25,7,3,8,1,0,0,0,2,false,true,false
6,3,3,0,9,11,0,10,0,2,0,0,0,0,0,12,12,9,3,6,4,0,0,0,1,false,false,false
7,1,4,3,11,8,6,0,12,8,1,1,0,0,0,27,27,11,37,35,7,2,2,1,1,true,false,true
8,3,4,1,10,11,0,11,0,5,0,0,0,0,0,13,13,17,9,13,2,0,0,1,1,false,false,false
9,0,2,2,5,0,10,14,8,8,0,0,0,0,0,9,9,16,13,11,8,1,0,1,4,true,false,true


In [6]:

rf_res = MLJ.predict(rf_machine, X)


4120-element CategoricalDistributions.UnivariateFiniteVector{OrderedFactor{2}, Bool, UInt32, Float64}:
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>0.769, true=>0.231)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>0.846, true=>0.154)
 UnivariateFinite{OrderedFactor{2}}(false=>0.462, true=>0.538)
 UnivariateFinite{OrderedFactor{2}}(false=>0.154, true=>0.846)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>0.615, true=>0.385)
 ⋮
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>0.0769, true=>0.923)
 UnivariateFinite{OrderedFactor{2}}(false=>

In [37]:
import Catan: PlayerType, DefaultRobotPlayer, read_map
players = Vector{PlayerType}([
    CatanLearning.EmpathRobotPlayer(:Blue, configs), 
    DefaultRobotPlayer(:Test2, configs),
    DefaultRobotPlayer(:Test3, configs),
    DefaultRobotPlayer(:Test4, configs),
   ])
player = players[1]
board = Board(configs)


Board(Dict(:N => 9, :F => 11, :M => 7, :A => 6, :D => 3, :R => 4, :B => 8, :Q => 6, :P => 5, :I => 3…), Dict(5 => [:P, :O], 7 => [:M], 12 => [:G], 8 => [:B, :C], 4 => [:R, :H], 6 => [:A, :Q], 11 => [:F, :L], 10 => [:S, :E], 2 => [:J], 9 => [:K, :N]…), Dict(:N => :Brick, :F => :Grain, :M => :Wood, :A => :Desert, :D => :Grain, :R => :Brick, :B => :Grain, :Q => :Wood, :P => :Brick, :I => :Pasture…), Dict{Tuple, Catan.Building}(), Dict{Tuple, Set{Road}}(), Dict{Tuple, Symbol}((1, 2) => :All, (6, 5) => :All, (2, 8) => :All, (4, 2) => :Grain, (6, 2) => :Brick, (1, 4) => :Pasture, (5, 9) => :Wood, (6, 4) => :All, (5, 8) => :Wood, (3, 2) => :Stone…), Any[[nothing, nothing, nothing, nothing, nothing, nothing, nothing], [nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], [nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], [nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothi

In [38]:
p_public = PlayerPublicView(player)
f = CatanLearning.compute_public_features(board, p_public)
#CatanLearning.predict_public_model(rf_machine, board, p_public)


21-element Vector{Pair{Symbol, Float64}}:
               :CountSettlement => 0.0
          :CountTotalSettlement => 0.0
                     :CountCity => 0.0
                     :CountRoad => 0.0
             :SumWoodDiceWeight => 0.0
            :SumBrickDiceWeight => 0.0
          :SumPastureDiceWeight => 0.0
            :SumStoneDiceWeight => 0.0
            :SumGrainDiceWeight => 0.0
                 :CountPortWood => 0.0
                                ⋮
                :CountPortStone => 0.0
                :CountPortGrain => 0.0
       :CountDevCardsUsedKnight => 0.0
     :CountDevCardsUsedMonopoly => 0.0
 :CountDevCardsUsedYearOfPlenty => 0.0
 :CountDevCardsUsedRoadBuilding => 0.0
                :HasLargestArmy => 0.0
                :HasLongestRoad => 0.0
      :CountVisibleVictoryPoint => 0.0

In [31]:
X_new = DataFrame(f)
CatanLearning.coerce_feature_types!(X_new)
predict(rf_machine, X_new)

BoundsError: BoundsError: attempt to access 21-element Vector{Int8} at index [23]

In [34]:
length(names(X_new))
length(names(df))

27

In [ ]:
#MLJ.predict(naive_machine, X)
#MLJ.save(naive_machine, "../CatanLearning/data/naive_public_model.jls")
predict(rf_machine, f)

In [10]:
mach = naive_machine
typeof(mach)

Machine{NaivePublicVictoryPoints, NaivePublicVictoryPoints, true}

In [11]:
typeof(mach.args[1])

MLJBase.Source